<font size=5 color="black">**任務流程**</font>
1. 前置作業 ＝> import 需要套件、建立相關身份認證資料
2. 打開開發人員工具，找到搜尋html，使用selenium輸入關鍵字搜尋，並進入搜尋結果頁面（詳情：selenium-dcard應用），104為動態網頁，使用selenium進行頁面滑動，依個人需求取得各職缺網址，並存成一個網址list
4. 透過迴圈進入各職缺網址內，取得相關資料，append進入output_list中，後續將此list轉成df
5. 將爬蟲回來的資料存成df，進行data cleaning，完成後輸出為excel檔！

參考資料:
<br>
1. [104人力銀行 網路爬蟲](https://tlyu0419.github.io/2020/06/19/Crawler-104HumanResource/)
<br>
2. [[網頁爬蟲] 104人力銀行標籤抓不到內容](https://ithelp.ithome.com.tw/questions/10198403)
<br>
3. [Python 使用 Beautiful Soup 抓取與解析網頁資料，開發網路爬蟲教學](https://blog.gtwang.org/programming/python-beautiful-soup-module-scrape-web-pages-tutorial/2/)

### 前置作業 ＝> import 需要套件、建立headers&相關cookies資料
1. 設置optins => 擋掉跳出視窗
2. 建立headers
3. 準備參數cookies (全職、關鍵字、清單條例式)

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import time
import re

In [ ]:
options = Options()
options.add_argument("--disable-notifications")

url_search = "https://104.com.tw/jobs/search/?"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
my_params = {'ro': '1',                # 限定全職的工作，如果不限定則輸入0
             'keyword': '數據分析',     # 想要查詢的關鍵字
             'mode': 'l'}              # 清單瀏覽模式


### 打開開發人員工具，找到搜尋html，使用selenium輸入關鍵字搜尋，並進入搜尋結果頁面

1. session連線 => 帶著my_params通過搜尋頁面，抵達搜尋結果頁面，104為動態網頁，使用selenium進行頁面滑動
2. 輸入欲爬取至第幾頁面
3. 建立url的beautiful soup物件，取得各職缺網址並存成work_url_lists
    - 使用**driver.page_source**來取得所有頁面資訊
    - 過濾掉置頂徵才廣告 => url中jobsource參數等於hotjob_chr者
    - 取得url中的職缺編號，並組成app版網址 =>　https://m.104.com.tw/job/ + 職缺編號(ex. 5944x)

In [ ]:
# session 連線
# 帶著參數通過搜尋頁面，抵達結果頁面的網址
ss = requests.session()
res = ss.get(url_search, headers=headers, params=my_params)

# 104為動態網頁，使用selenium進行頁面滑動，蒐集網址
# 輸入欲爬取至第幾頁面
page_number = int(input("請輸入欲爬取至第幾頁面: "))
driver = Chrome("../../chromedriver", options=options)
driver.get(res.url)
for i in range(1, page_number+1):
    driver.execute_script("var s = document.documentElement.scrollTop=5000")
    print("目前爬至第{}頁".format(i))
    time.sleep(5)

# 建立beautiful soup物件
# 使用driver.page_source來取得所有頁面資訊
# 過濾掉置頂徵才廣告 => url中包含hotjob_chr參數
# 取得url中的職缺編號，並組成手機版網址 =>　https://m.104.com.tw/job/ + 職缺編號(ex. 5944x)
soup = BeautifulSoup(driver.page_source, features='html.parser')
url_data = [url["href"] for url in soup.select('li.job-mode__jobname a') if url["href"].split("=")[1] != 'hotjob_chr']
work_url_lists = list(map(lambda url: 'https://m.104.com.tw/job/{}'.format(re.findall("job/(.*)\?", url)[0]), url_data))

# 關閉頁面
driver.close()

In [ ]:
print(work_url_lists)

### 透過迴圈進入各職缺網址內，取得相關資料，append進入output中，後續將此轉成df
1. 每3秒爬一次，避免被鎖ip或是重導到電腦版網址，因而產生錯誤
2. 取得職缺相關資料
    - 更新時間、公司、職位、工作地點、工作內容
    - 工作需求技能、工具、其他條件
    - 額外資訊(聯絡人、薪水)
5. append進入output

In [ ]:
# 每3秒爬一次，避免被鎖或是導向網頁，造成錯誤發生
output = []
for work_url in work_url_lists:
    print(work_url)
    res_work = ss.get(work_url, headers=headers, allow_redirects=False)
    w_soup = BeautifulSoup(res_work.text, 'html.parser')
    
    try:
        # 更新時間、公司、職位、工作地點、工作內容
        update_time = w_soup.select("time")[0]['datetime']
        company = w_soup.select('h2.company')[0].text
        title = w_soup.select('h1.title')[0].text
        work_place = w_soup.select('div.content a.addr')[0].text.strip()
        description = w_soup.select("div.content p")[0].text.strip()
        
        # 工作需求技能、工具、其他條件
        job_skill = " ".join([i.text for i in w_soup.select("a[data-gtm='job-skill']")])
        job_tool = " ".join([i.text for i in w_soup.select("a[data-gtm='job-tool']")])
        if w_soup.select("td div.cut"):
            plus = w_soup.select("td div.cut")[0].text
        else:
            plus = ""
        
        ### 額外資訊(聯絡人、薪水)
        recruiter = w_soup.find_all("table", class_="column2 contact")[0].select("td")[0].text
        extra_data = ''.join([t.text for t in w_soup.select("table.column2 td")])
        salary = re.findall(r"待遇面議.*|[年月]薪.*[元以上]", extra_data) 
        
    except ValueError as e:
        print("Value error", work_url)
    except IndexError as e:
        print("Index error", work_url)
        
    output.append([update_time, company, title, work_place, salary, description, job_skill, job_tool, plus, recruiter, work_url])
    
    time.sleep(3)


### 將爬蟲回來的資料存成df，進行data cleaning
1. 設置欄位
2. 去除重複值 => 透過company, title, work_place來篩選，保留第一個(日期較新)
3. 建立將df備份成df2，避免資料處理中不小心改動到原檔因而得重新來過

In [ ]:
df = pd.DataFrame(output, columns=['update_time', 'company', 'title', 'work_place', 'salary', 'description', 'job_skill', 'job_tool', "plus", "recruiter", "URL"])

In [ ]:
df.shape

In [ ]:
df2 = df.copy()

#### 去除重複值 => 透過company, title, work_place來篩選掉相同重複值缺，保留第一個(日期較新)

In [ ]:
df2.drop_duplicates(subset=['company', 'title', 'work_place'], keep="first", inplace=True)
df2.sort_values(by="update_time", ascending=False, inplace=True)
df2

#### 薪資(salary)
將薪資進行統一格式處理
- 分成下限 & 上限 => 以利後續查找資料
- 年薪改為月薪 => 除以12
- 待遇面議改為下限4w、上限面議



In [ ]:
df2["salary"] = df2["salary"].apply(lambda x: re.sub(" ", "", x[0]))

In [ ]:
df2.loc[(df2["salary"].str.contains("待遇面議")), "salary"] = "40000~0"

In [ ]:
df2["salary"] = df2["salary"].apply(lambda x: re.sub(",|[^0-9~]", "", x)) 

In [ ]:
# 有些只寫起薪的會造成error，index用-1避免報錯
df2["salary_min"] = df2["salary"].map(lambda x: x.split("~")[0])
df2["salary_max"] = df2["salary"].map(lambda x: x.split("~")[-1])

In [ ]:
# data type轉成 numeric
df2[["salary_min", "salary_max"]] = df2[["salary_min", "salary_max"]].apply(pd.to_numeric)

In [ ]:
## 年薪改為月薪 => 此處用平均數加上3倍標準差來過濾出極端值後進行處理，應該有更好的方法
cond_low = df2["salary_min"].mean() + 3*df2["salary_min"].std()
cond_upper = df2["salary_max"].mean() + 3*df2["salary_max"].std()

df2["salary_min"] = df2["salary_min"].apply(lambda x: int(x/12) if x > cond_low else x)
df2["salary_max"] = df2["salary_max"].apply(lambda x:int(x/12) if x >cond_upper else x)

In [ ]:
# 只寫起薪或salary_max等於0者，設定為"面議"
df2.loc[(df2["salary_min"] == df2["salary_max"]) | (df2["salary_max"] == 0), "salary_max"] = "面議"

In [ ]:
df2

#### 地區
1. 去除所有空格
2. 從work_place中分成出area

In [ ]:
df2["work_place"] = df2["work_place"].apply(lambda x: re.sub(" ", "", x))

In [ ]:
df2["area"] = df2["work_place"].str.strip().str[:3]

In [ ]:
df2["work_place"] = df2["work_place"].str.strip().str[3:]

#### 處理job_skill 、 job_tool

- 為空值者，設為不拘 


In [ ]:
df2.loc[(df2["job_tool"] == "") | (df2["job_skill"] == ""), ["job_skill", "job_tool"]] = "不拘"

#### 其他條件(plus)、工作內容(description) => 除去任何空白字符
- 其他條件(plus)為空白者，設為無

In [ ]:
df2["plus"] = df2["plus"].apply(lambda x: re.sub(r"\s", "", x)) ## r"\s" => r""表示此字串套用re規則，\s代表匹配任何空白字符

In [ ]:
df2.loc[(df2["plus"] == ""), "plus"] = "無"

In [ ]:
df2["description"] = df2["description"].apply(lambda x: re.sub(r"\s", "", x))

In [ ]:
df2

#### 重新排序columns

- 欄位重新排為以下順序 => 'update_time', 'company', 'title', 'area', 'work_place', 'salary_min', 'salary_max', 'description', 'job_skill', 'job_tool', 'plus', 'recruiter', 'URL'

In [ ]:
col = list(df2.columns)
col

In [ ]:
df2 = df2[col[:3] + [col[-1]] + [col[3]] + col[-3:-1] + col[5:-3]]

In [ ]:
df2

#### 重設index

In [ ]:
df2.reset_index(drop=True, inplace=True)

#### 存成excel檔

In [ ]:
df2.to_excel("./104_job_bank(data analysis related).xlsx", index=False)